In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

# Establish client connection
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Bulk insert of courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Courses inserted successfully.')

Courses inserted successfully.


In [7]:
# Query for courses with enrollments greater than 20
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('690c4311d2c1e0d14643a57b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c4311d2c1e0d14643a57c'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690c44e0d2c1e0d14643a57f'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c44e0d2c1e0d14643a580'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690c44e0d2c1e0d14643a583'), 'course': 'Math 202', 'enrollments': 35, 'department': 'Mathematics'}
{'_id': ObjectId('690c44e0d2c1e0d14643a584'), 'course': 'CS 201', 'enrollments': 22, 'department': 'Computer Science'}
{'_id': ObjectId('690c44e0d2c1e0d14643a586'), 'course': 'History 301', 'enrollments': 30, 'department': 'History'}
{'_id': ObjectId('690c44e0d2c1e0d14643a588'), 'course': 'Physics 303', 'enrollments': 28, 'department': 'Physics'}
{'_id': ObjectId('690c44e0d2c1e0d14643a589'), 'course': 'Chemistr

In [7]:
# Query courses in Computer Science or Mathematics departments
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematics']}}):
    print(course)


{'_id': ObjectId('690c4311d2c1e0d14643a57b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690c4311d2c1e0d14643a57c'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [8]:
# Average enrollment per department using aggregation
pipeline = [
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)


{'_id': 'English', 'average_enrollment': 50.0}
{'_id': 'Physics', 'average_enrollment': 18.25}
{'_id': 'Computer Science', 'average_enrollment': 23.0}
{'_id': 'Mathematics', 'average_enrollment': 31.25}
{'_id': 'History', 'average_enrollment': 21.0}
{'_id': 'Chemistry', 'average_enrollment': 40.0}
{'_id': 'Biology', 'average_enrollment': 45.0}
{'_id': 'Philosophy', 'average_enrollment': 12.0}


In [9]:

# Maximum enrollment per department
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'English', 'max_enrollment': 50}
{'_id': 'Physics', 'max_enrollment': 28}
{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'Mathematics', 'max_enrollment': 35}
{'_id': 'History', 'max_enrollment': 30}
{'_id': 'Chemistry', 'max_enrollment': 40}
{'_id': 'Biology', 'max_enrollment': 45}
{'_id': 'Philosophy', 'max_enrollment': 12}


In [10]:
# Projection to rename fields
pipeline = [
    {'$project': {'course_name': '$course', 'department_name': '$department', 'enrollments': 1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c4311d2c1e0d14643a57b'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690c4311d2c1e0d14643a57c'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690c4311d2c1e0d14643a57d'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('690c4311d2c1e0d14643a57e'), 'enrollments': 15, 'course_name': 'Physics 202', 'department_name': 'Physics'}
{'_id': ObjectId('690c44e0d2c1e0d14643a57f'), 'enrollments': 30, 'course_name': 'Math 101', 'department_name': 'Mathematics'}
{'_id': ObjectId('690c44e0d2c1e0d14643a580'), 'enrollments': 25, 'course_name': 'CS 102', 'department_name': 'Computer Science'}
{'_id': ObjectId('690c44e0d2c1e0d14643a581'), 'enrollments': 20, 'course_name': 'History 201', 'department_name': 'History'}
{'_id': ObjectId('690c44e0d2c1e0d14643a582'), 'enrollments': 15, 'course_name': 'Physics 202', 'department_name': '

In [11]:
# Adding enrollment category field based on enrollments
pipeline = [
    {'$addFields': {'enrollment_category': {'$cond': {'if': {'$gt': ['$enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('690c4311d2c1e0d14643a57b'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690c4311d2c1e0d14643a57c'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690c4311d2c1e0d14643a57d'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'low'}
{'_id': ObjectId('690c4311d2c1e0d14643a57e'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'low'}
{'_id': ObjectId('690c44e0d2c1e0d14643a57f'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'high'}
{'_id': ObjectId('690c44e0d2c1e0d14643a580'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'high'}
{'_id': ObjectId('690c44e0d2c1e0d14643a581'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', '

In [2]:
course_collections = db['courses']

In [7]:
# Bulk insert of additional courses with student enrollments
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 27, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}),
    
    # New courses
    InsertOne({'course': 'Math 202', 'enrollments': 35, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 201', 'enrollments': 29, 'department': 'Computer Science'}),
    InsertOne({'course': 'CS 301', 'enrollments': 28, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 301', 'enrollments': 30, 'department': 'History'}),
    InsertOne({'course': 'History 404', 'enrollments': 15, 'department': 'History'}),
    InsertOne({'course': 'Physics 303', 'enrollments': 28, 'department': 'Physics'}),
    InsertOne({'course': 'Chemistry 101', 'enrollments': 40, 'department': 'Chemistry'}),
    InsertOne({'course': 'Biology 110', 'enrollments': 45, 'department': 'Biology'}),
    InsertOne({'course': 'Philosophy 101', 'enrollments': 12, 'department': 'Philosophy'}),
    InsertOne({'course': 'English 200', 'enrollments': 50, 'department': 'English'})
]

# Perform bulk insert into the courses collection
course_collections.bulk_write(operations)
print('Courses inserted successfully.')



Courses inserted successfully.


In [8]:
# Aggregation to get count of courses per department
pipeline =[
    {"$group" : {"_id": "$department", 
                 "courses_count": {"$sum":1}
                }
    }
]

for result in course_collections.aggregate(pipeline):
    print(result)


{'_id': 'Philosophy', 'courses_count': 1}
{'_id': 'History', 'courses_count': 3}
{'_id': 'Mathematics', 'courses_count': 2}
{'_id': 'Computer Science', 'courses_count': 3}
{'_id': 'Chemistry', 'courses_count': 1}
{'_id': 'Biology', 'courses_count': 1}
{'_id': 'Physics', 'courses_count': 2}
{'_id': 'English', 'courses_count': 1}


In [9]:
# Aggregation to get courses with Enrollments > 25 in Computer Science
pipeline = [
    {"$match": {"department":"Computer Science","enrollments": {"$gt": 25}}}
    
      
]

for filtering in course_collections.aggregate(pipeline):
    print(filtering)

{'_id': ObjectId('690c503b5164a175b3296909'), 'course': 'CS 102', 'enrollments': 27, 'department': 'Computer Science'}
{'_id': ObjectId('690c503b5164a175b329690d'), 'course': 'CS 201', 'enrollments': 29, 'department': 'Computer Science'}
{'_id': ObjectId('690c503b5164a175b329690e'), 'course': 'CS 301', 'enrollments': 28, 'department': 'Computer Science'}


In [10]:
# Aggregation to get courses with enrollments > 25 in Computer Science
pipeline = [
    # Match documents for "Computer Science" department and enrollments > 25
    {"$match": {
        "department": "Computer Science",
        "enrollments": {"$gt": 25}
        }
    },
    
    # Group by course and department, and get the first enrollment for each group
    {"$group": {"_id":{
                "Course": "$course",
                "Department": "$department"},
                "Enrollments": {"$first": "$enrollments"}  # First enrollment value for each group
               }
    }
]


# Execute aggregation
for filtering in course_collections.aggregate(pipeline):
    print(filtering)

{'_id': {'Course': 'CS 301', 'Department': 'Computer Science'}, 'Enrollments': 28}
{'_id': {'Course': 'CS 102', 'Department': 'Computer Science'}, 'Enrollments': 27}
{'_id': {'Course': 'CS 201', 'Department': 'Computer Science'}, 'Enrollments': 29}


In [73]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.3 MB/s  0:00:00m 4.3 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import random
import faker

fake = faker.Faker()
students_collection = db['students']

course_enrollments = [
    {"course": "Math 101", "enrollments": 30},
    {"course": "CS 102", "enrollments": 27},
    {"course": "History 201", "enrollments": 20},
    {"course": "Physics 202", "enrollments": 15},
    {"course": "Math 202", "enrollments": 35},
    {"course": "CS 201", "enrollments": 29},
    {"course": "CS 301", "enrollments": 28},
    {"course": "History 301", "enrollments": 30},
    {"course": "History 404", "enrollments": 15},
    {"course": "Physics 303", "enrollments": 28},
    {"course": "Chemistry 101", "enrollments": 40},
    {"course": "Biology 110", "enrollments": 45},
    {"course": "Philosophy 101", "enrollments": 12},
    {"course": "English 200", "enrollments": 50}
]

students_operations = []

# Generate 5 students for each course
for course in course_enrollments:
    for _ in range(course['enrollments']):  # Generate number of students based on enrollments
        student_name = fake.first_name()
        student_id = random.randint(1000, 9999)  # Random student ID
        students_operations.append(InsertOne({
            'student_id': student_id,
            'name': student_name,
            'course_enrollment': course['enrollments']  # Match to course enrollments
        }))
        
students_collection.bulk_write(students_operations)

print(f"Inserted {len(students_operations)} dummy student records into the students collection.")

Inserted 404 dummy student records into the students collection.


In [15]:
for i in students_collection.find():
    print(i)

{'_id': ObjectId('690c50645164a175b3296916'), 'student_id': 2073, 'name': 'Michael', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b3296917'), 'student_id': 5905, 'name': 'Amanda', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b3296918'), 'student_id': 8669, 'name': 'Stephanie', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b3296919'), 'student_id': 3227, 'name': 'Mario', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b329691a'), 'student_id': 9045, 'name': 'Jack', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b329691b'), 'student_id': 6340, 'name': 'Courtney', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b329691c'), 'student_id': 4130, 'name': 'Joseph', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b329691d'), 'student_id': 3738, 'name': 'Jose', 'course_enrollment': 30}
{'_id': ObjectId('690c50645164a175b329691e'), 'student_id': 9682, 'name': 'Jeremy', 'course_enrollment': 30}
{'_id': ObjectId('

In [14]:
# Define the aggregation pipeline to join courses with students
pipeline = [
    {
        "$lookup": {
            "from": "students",  # The students collection
            "localField": "enrollments",  # The field in courses to match
            "foreignField": "course_enrollment",  # The field in students to match
            "as": "students"  # The name of the array that will hold the matched student documents
        }
    },
    {
        "$unwind": {
            "path": "$students",  # Unwind the 'students' array to get one document per match
            "preserveNullAndEmptyArrays": True  # If no match is found, include the course with null student data
        }
    },
    {
        "$project": {
            "_id": 0,  # Optionally exclude the _id field
            "course": 1,
            "department": 1,
            "enrollments": 1,
            "student_name": "$students.name",  # Assuming `name` is a field in the students collection
            "student_id": "$students.student_id"  # Assuming `student_id` is a field in the students collection
        }
    }
]

# Execute the aggregation pipeline
for filtering in course_collections.aggregate(pipeline):
    print(filtering)
    

{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Michael', 'student_id': 2073}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Amanda', 'student_id': 5905}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Stephanie', 'student_id': 8669}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Mario', 'student_id': 3227}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Jack', 'student_id': 9045}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Courtney', 'student_id': 6340}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Joseph', 'student_id': 4130}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'student_name': 'Jose', 'student_id': 3738}
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathem